In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Cargar los datos de train y test (asegúrate de tener los archivos o adaptar el código)
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Suponiendo que 'price_in_euros' es la variable objetivo
X = train.drop("price_in_euros", axis=1)
y = train["price_in_euros"]

# Dividir el set de train en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Pipeline para variables numéricas: imputación, transformación para normalizar y escalado
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("pt", PowerTransformer(method="yeo-johnson")),  # Ayuda a normalizar la distribución
    ("scaler", StandardScaler())
])

# Pipeline para variables categóricas: imputación y codificación one-hot
categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combinar ambos pipelines en un preprocesador
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_pipeline, num_cols),
    ("cat", categorical_pipeline, cat_cols)
])

# Definir el modelo: XGBoost Regressor
xgb_reg = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# Crear el pipeline completo: preprocesado + modelo
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", xgb_reg)
])

# Definir el espacio de hiperparámetros a explorar
param_grid = {
    "regressor__n_estimators": [100, 300, 500],
    "regressor__max_depth": [3, 5, 7],
    "regressor__learning_rate": [0.01, 0.05, 0.1],
    "regressor__subsample": [0.7, 0.8, 1.0],
    "regressor__colsample_bytree": [0.7, 0.8, 1.0]
}

# Definir el scorer usando RMSE (se utiliza un lambda para calcular la raíz del MSE)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

# Configurar la validación cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Búsqueda aleatoria de hiperparámetros
random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=30,
                                   scoring=rmse_scorer, cv=cv, verbose=1,
                                   random_state=42, n_jobs=-1)

# Ajustar el modelo con el set de entrenamiento
random_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros y el score obtenido
print("Mejores hiperparámetros:", random_search.best_params_)
print("Mejor RMSE (score negativo):", random_search.best_score_)

# Evaluar el desempeño en el set de validación
y_pred_val = random_search.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print("RMSE en validación:", rmse_val)

# Realizar predicciones sobre el set de test
predictions_test = random_search.predict(test)

# Guardar las predicciones (asumiendo que el test tiene una columna 'id' o se generan índices)
output = pd.DataFrame({
    "id": test["id"] if "id" in test.columns else np.arange(len(test)),
    "price_in_euros": predictions_test
})
output.to_csv("submission.csv", index=False)
